In [8]:
#@ Importing Necessary dependencies and libraries:
from nltk.tag import pos_tag
!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer, confusion_matrix
from pprint import pprint
from sklearn.metrics import f1_score, classification_report
from sklearn.pipeline import Pipeline
import string
import warnings
warnings.filterwarnings('ignore')

In [9]:
#@ Data Loading:
def load_data_conll(file_path):
  myoutput, tokens, tags=[], [], []
  with open(file_path, 'r') as fh:
    for line in fh:
      line=line.strip() #to remove leading and trailing  white space characters
      if '\t' not in line:
        #sentences ended
        myoutput.append([tokens, tags])
        tokens, tags=[], []
      else:
        token ,tag=line.split('\t')
        tokens.append(token)
        tags.append(tag)
    fh.close()
    return myoutput

### Getting features of all the tokens in sentence.

##### Features:
- **Token Context:** a window of 2 tokens on either side of current token, and current token.

- **POS Context:**  a window of 2 tokens on either side of current tag, and current tag.


In [10]:
def sentence2features(sentence):
  features=[]
  sentence_tags=pos_tag(sentence)
  for i in range(0, len(sentence)):
    token=sentence[i]
    tokenfeatures={}

  #@ token features:
   #token
  tokenfeatures['token']=token

  #for 2 prev tokens
  if i==0:
    tokenfeatures['prevtoken']=tokenfeatures['prevsecondtoken']='<S>'
  elif i==1:
    tokenfeatures['prevtoken']=sentence[0]
    tokenfeatures['prevsecondtoken']='</S>'
  else:
    tokenfeatures['prevtoken']=sentence[i-1]
    tokenfeatures['prevsecondtoken']=sentence[i-2]

  #for 2 next token
  if i==len(sentence)-2:
    tokenfeatures['nexttoken']=sentence[i+1]
    tokenfeatures['nextsecondtoken']='</S>'

  elif i==len(sentence)-1:
     tokenfeatures['nexttoken']='</S>'
     tokenfeatures['nextsecondtoken']='</S>'

  else:
    tokenfeatures['nexttoken']=sentence[i+1]
    tokenfeatures['nextsecondtoken']=sentence[i+2]

  #@ POS feature:

  #current tag
  tokenfeatures['tag']=sentence_tags[i][1]

  #prev tag
  if i==0:
    tokenfeatures['prevtag']=tokenfeatures['prevsecondtag']='</S>'
  elif i==1:
    tokenfeatures['prevtag']=sentence_tags[0][1]
    tokenfeatures['prevsecondtag']='</S>'
  else:
     tokenfeatures['prevtag']=sentence_tags[i-1][1]
     tokenfeatures['prevsecondtag']=sentence_tags[i-2][1]

  #next tag
  if i==len(sentence)-2:
     tokenfeatures['nexttag']=sentence_tags[i+1][1]
     tokenfeatures['nextsecondtag']='</S>'

  elif i==len(sentence)-1:
    tokenfeatures['nexttag']='</S>'
    tokenfeatures['nextsecondtag']='</S>'

  else:
     tokenfeatures['nexttag']=sentence_tags[i+1][1]
     tokenfeatures['nextsecondtag']=sentence_tags[i+2][i]

  features.append(tokenfeatures)
  return features

In [11]:
#@ Extracting features:
def get_features_conll(conll_data):
  features=[]
  labels=[]
  for sentence in conll_data:
    features.append(sentence2features(sentence[0]))
    labels.append(sentence[1])
  return features, labels